### Objective: 
#### Given a large labeled source dataset with annotations for a task set, A, we seek to transfer knowledge to a sparsely labeled target domain with a possibly wholly new task set, B.

# Experiment :1 --> 0-4 SVNH Subset to 5-9 MNIST


### Data Preparation 

In [4]:
import torch
#print(torch.__version__) # torch Version
import numpy as np
from torchvision.datasets import MNIST,SVHN
from torch.utils.data import Dataset, DataLoader
from torch.tensor import Tensor as T

# To Do:
# Filter the Dataset Based on Range of Labels Given
class CustomMNIST(MNIST):
    """
    Additional Args:
        min_label : Minimun Value of Label That is to be allowed. 
        # Todo : Need to support list of values instead of min value
    """
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False,min_label=5):
        super().__init__(root, train=train, transform=transform, target_transform=target_transform, download=download)
        self.min_label=min_label
        self.data=self.data[self.targets>=self.min_label]
        self.targets=self.targets[self.targets>=self.min_label]


# Checked And working as expected
# mnist=FilteredMnist('.',download=True,label_min=5)
# print((mnist.train_labels))

class CustomSVNH(SVHN):
    def __init__(self, root, split='train', transform=None, target_transform=None, download=False,max_label=5):
        super().__init__(root, split=split, transform=transform, target_transform=target_transform, download=download)
        self.max_label=max_label
        self.data=self.data[self.labels<=self.max_label]
        self.labels=self.labels[self.labels<=self.max_label]


svnh_train=CustomSVNH(root='~/.torch',download=True,max_label=4)
svnh_test=CustomSVNH(root='~/.torch',split='test',download=True,max_label=4)

mnist_train=CustomMNIST(root='~/.torch',download=True,min_label=5)
mnist_test=CustomMNIST(root='~/.torch',train=False,download=True,min_label=5)


# print(svnh.data.shape,svnh.labels.shape)
#print(np.unique(svnh.labels))
#print(np.unique(mnist.train_labels))
print("Train",svnh_train.data.shape,svnh_train.labels.shape,"Test:",svnh_test.data.shape,svnh_test.labels.shape)
print("Train",mnist_train.data.shape,mnist_train.targets.shape,"Test:",mnist_test.data.shape,mnist_test.targets.shape)


Using downloaded and verified file: C:\Users\Gokkulnath/.torch\train_32x32.mat
Using downloaded and verified file: C:\Users\Gokkulnath/.torch\test_32x32.mat
Train (45349, 3, 32, 32) (45349,) Test: (16397, 3, 32, 32) (16397,)
Train torch.Size([29404, 28, 28]) torch.Size([29404]) Test: torch.Size([4861, 28, 28]) torch.Size([4861])


## Data has been Filtered as Required 

![](Architecture%20Modelling.PNG) 

In [ ]:
# Todo : 
- Decide whether to add Common Layers or Select them from existing model definitions based on depth?
- How Multiple inputs are fed to LabelEfficientModel? [inp1,inp2,inp3]?


# Model

In [ ]:
base_model=nn.Sequential(
[nn.Conv2d(in_channels=3,out_channels=32)

])

In [ ]:
class CustomModel(nn.Module):
    def __init__(self,num_classes=5):
        super(CustomModel,self).__init__()
        
        # prep :
        self.prep = nn.Sequential(
                            nn.Conv2d(in_channels=3,out_channels=,) 
        
        
                            )
        self.layer1 = nn.Sequential(
        
                        )
        self.layer2 = nn.Sequential(
        
        
                    )
        self.layer2 = nn.Sequential(
        
        
                    )
        self.classifier = nn.Sequential(
                    nn.AdaptiveMaxPool2d(64),
                    nn.Flatten()
                    nn.Linear(in_features=64,out_features=num_classes)        
                    )
        
        # Follow Resnet9 ?
        
        

In [9]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim

class LabelEfficientModel(nn.Module):
    
    def __init__(base_model,n_common_layers,*args,**kwargs):
        pass
    def __call__():
        pass
    

# Softmax With Temperature

- Temperature is a hyperparameter used to control the randomness of predictions by scaling the logits before applying softmax. Temperature represents how much to divide the logits by before computing the softmax. 
* When the temperature is 1, we compute the softmax directly on the logits (the unscaled output of earlier layers), and using a temperature of 0.6 the model computes the softmax on $logits$/$0.6$, resulting in a larger value

Performing softmax on larger values makes the LSTM more confident (less input is needed to activate the output layer) but also more conservative in its samples (it is less likely to sample from unlikely candidates). Using a higher temperature produces a softer probability distribution over the classes, and makes the RNN more “easily excited” by samples, resulting in more diversity and also more mistakes.

Neural networks produce class probabilities with logit vector z where z=(z1,…,zn) by performing the softmax function to produce probability vector q=(q1,…,qn) by comparing zi with with the other logits.

qi=exp(zi/T)∑jexp(zj/T)(1)
where T is the temperature parameter, normally set to 1.

The softmax function normalizes the candidates at each iteration of the network based on their exponential values by ensuring the network outputs are all between zero and one at every timestep.

Temperature therefore increases the sensitivity to low probability candidates. In LSTMs, the candidate, or sample, can be a letter, a word, or musical note, for example:

Source : 
- https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally
- Hinton, Geoffrey, Oriol Vinyals, and Jeff Dean. "Distilling the knowledge in a neural network." arXiv preprint arXiv:1503.02531 (2015). arXiv




In [6]:
# utils

def softmax_temperature(logits,temperature=1.0):
    s=torch.exp(logits/temperature)
    return  s/torch.sum(s)
# np.exp(logits/temperature)#/
    

In [7]:
a=T([0.1,0.3,0.4,0.5,0.15])

In [8]:
for i in np.arange(0.01,1.0,0.1):
    print(i,softmax_temperature(a,i))

0.01 tensor([4.2482e-18, 2.0611e-09, 4.5398e-05, 9.9995e-01, 6.3048e-16])
0.11 tensor([0.0161, 0.0994, 0.2467, 0.6123, 0.0254])
0.21000000000000002 tensor([0.0635, 0.1646, 0.2649, 0.4265, 0.0806])
0.31000000000000005 tensor([0.0966, 0.1842, 0.2544, 0.3512, 0.1136])
0.41000000000000003 tensor([0.1178, 0.1918, 0.2448, 0.3125, 0.1331])
0.51 tensor([0.1320, 0.1954, 0.2377, 0.2892, 0.1456])
0.6100000000000001 tensor([0.1421, 0.1973, 0.2324, 0.2738, 0.1543])
0.7100000000000001 tensor([0.1497, 0.1984, 0.2284, 0.2629, 0.1606])
0.81 tensor([0.1555, 0.1991, 0.2252, 0.2548, 0.1654])
0.91 tensor([0.1601, 0.1995, 0.2227, 0.2485, 0.1692])


##### Baselines:
* Labelled Data in Source Domain D1
* few labeled data in target domain D2
* unlabeled data in target domain D3
* Since our approach involves using annotations from a small subset of the data, we randomly subsample 10 different subsets from D2 from the training split of MNIST dataset, and use the remaining data as D3 for each k.

P.S :To fairly compare with traditional learning paradigm and episodic training, we subsample k examples from each class to construct dataset D2 so that we can perform traditional training or episodic (k - 1)-shot learning. k = 2; 3; 4; 5, which corresponds to 10; 15; 20; 25 labeled examples

- Target Only:
- Fine Tune model Pretrained on D1 --> D2
- Matching networks: pretrain the model on D3, then use D2 as the support set in the matching networks;
- Fine-tuned matching networks: same as baseline iii, except that for each k the model is fine-tuned on D2 with 5-way (k - 1)-shot learning: k 􀀀 1 examples in each class are randomly selected as the support set, and the last example in each class is used as the query set;
- Fine-tune + adversarial: in addition to baseline ii, the model is also trained on D1 and D3 with a domain adversarial loss;
- Full model: fine-tune the model with the proposed multi-layer domain adversarial loss.